# Catalog demand

A company that sells high end home goods is considering sending out this year as well its catalog. 
Given a dataset df with historical data, the task is to forecast the profit if the catalog is sent out to 250 new customers which information are contain in the dataframe mailing. 

The historical data will be used to build a linear regression model which will then be applied to the new customers. 
The first step in this analysis is to get a sense of the data, so to spot potential predictor variables, then the model itself will be built. 

## Data exploration

### Historical data

In [1]:
import pandas as pd

import statsmodels.api as sm

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.transform import jitter

import seaborn as sb

In [2]:
df = pd.read_excel('/Users/jackfederle/Documents/Business Analyst - Udacity/#1 Catalog demand/p1-customers.xlsx')
mailing = pd.read_excel('/Users/jackfederle/Documents/Business Analyst - Udacity/#1 Catalog demand/p1-mailinglist.xlsx')
df.head()

,Name,Customer_Segment,Customer_ID,Address,City,State,ZIP,Avg_Sale_Amount,Store_Number,Responded_to_Last_Catalog,Avg_Num_Products_Purchased,#_Years_as_Customer
0,Pamela Wright,Store Mailing List,2.0,376 S Jasmine St,Denver,CO,80224.0,227.90,100.0,No,1.0,6.0
1,Danell Valdez,Store Mailing List,7.0,12066 E Lake Cir,Greenwood Village,CO,80111.0,55.00,105.0,Yes,1.0,6.0
2,Jessica Rinehart,Store Mailing List,8.0,7225 S Gaylord St,Centennial,CO,80122.0,212.57,101.0,No,1.0,3.0
3,Nancy Clark,Store Mailing List,9.0,4497 Cornish Way,Denver,CO,80239.0,195.31,105.0,Yes,1.0,6.0
4,Andrea Brun,Store Mailing List,10.0,2316 E 5th Ave,Denver,CO,80206.0,110.55,100.0,Yes,1.0,2.0


Here it is already known that all the customers are from the same states and that there are 10 stores in total. 
Average sale amount is going to be the target variable of my model. 

There are a few variables that look promessing, which are: Customer Segment, Average number of products purchased, number of years as customers. The data about the location of the customers could be interesting, however, being all of them from the same state and shopping at a high end home goods store, I do not expect it to be significant. 
Responded to last catalog is for sure an interesting parameter to consider, but being the new customers new, catalogs were never sent to them. 

### New customers data

In [3]:
mailing.head()

,Name,Customer_Segment,Customer_ID,Address,City,State,ZIP,Store_Number,Avg_Num_Products_Purchased,#_Years_as_Customer,Score_No,Score_Yes
0,A Giametti,Loyalty Club Only,2213,5326 S Lisbon Way,Centennial,CO,80015,105,3,0.2,0.694964,0.305036
1,Abby Pierson,Loyalty Club and Credit Card,2785,4344 W Roanoke Pl,Denver,CO,80236,101,6,0.6,0.527275,0.472725
2,Adele Hallman,Loyalty Club Only,2931,5219 S Delaware St,Englewood,CO,80110,101,7,0.9,0.421118,0.578882
3,Alejandra Baird,Loyalty Club Only,2231,2301 Lawrence St,Denver,CO,80205,103,2,0.6,0.694862,0.305138
4,Alice Dewitt,Loyalty Club Only,2530,5549 S Hannibal Way,Centennial,CO,80015,104,4,0.5,0.612294,0.387706


The value Score_Yes (score_no = 1 - score_yes) is the probability of a customer actually buying after having received the catalog. It will be used at the end to estimate the expected revenue. Score_yes is the result of a classification model, like a logistic regression. I tried to compute it myself ex novo, however, with the data here at my disposal I could not put together a model decent enough, so I will just rely on the given data. 

In [4]:
mailing.describe()

,Customer_ID,ZIP,Store_Number,Avg_Num_Products_Purchased,#_Years_as_Customer,Score_No,Score_Yes
count,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000
mean,1764.148000,80117.780000,104.396000,4.672000,0.550400,0.659332,0.340668
std,943.734953,102.345797,2.612755,2.752949,0.302367,0.190043,0.190044
min,18.000000,80002.000000,100.000000,1.000000,0.000000,0.000002,0.185729
25%,949.250000,80014.000000,102.000000,3.000000,0.300000,0.612926,0.210961
50%,1882.000000,80121.000000,105.000000,5.000000,0.600000,0.742010,0.257990
75%,2499.500000,80219.750000,106.000000,6.000000,0.800000,0.789039,0.387074
max,3328.000000,80401.000000,109.000000,27.000000,1.000000,0.814271,0.999998


As said above, I can rule out responded to last catalog as potential predictor variable. Before I move on with the data visualisation, I am going to check the correlation among the potential predictors and the target variable

In [5]:
# correlation
correlation = df.corr(method='pearson')
print(correlation.loc['Avg_Sale_Amount'])

Customer_ID                   0.038235
ZIP                           0.007973
Avg_Sale_Amount               1.000000
Store_Number                 -0.007946
Avg_Num_Products_Purchased    0.855754
#_Years_as_Customer           0.029782
Name: Avg_Sale_Amount, dtype: float64


Number of years as customer can be ruled out as well. I am still going to include it in the visualisations just because a grid of four looks better than a grid of three

## Data visualisation

For data visualisaion I decided to use Bokeh, as it is a package with which I am currently becoming familiar. 
I find it particular interesting for the tools that can be integrated in the visualiations like the hover tool.

In [6]:
output_notebook()

Loading BokehJS ...

In [7]:
source = ColumnDataSource(df)

In [8]:
hover = HoverTool(tooltips=[('Customer Name', '@Name'),
                           ('Customer segment', '@Customer_Segment')])
p1 = figure(x_axis_label='Avg_Num_Products_Purchased', y_axis_label='Average sales amount', width=600)
p1.circle('Avg_Num_Products_Purchased', 'Avg_Sale_Amount', source=source, alpha=.6)
p1.add_tools(hover)

I'm still going to show the relation between average sales amount and number of years as customer because a grid of four looks better than a grid of three and because images stand out 

In [9]:
p2 = figure(x_axis_label='Number of Years as Customer', y_axis_label='Average sales amount', tools=[hover], width=600)
p2.circle('#_Years_as_Customer', 'Avg_Sale_Amount', source=source, alpha=.6)
p2.add_tools(hover)

Rapresenting categorical data with a scatterplot is second best, however I make up for it using some jitter and a low alpha.  

In [10]:
p3 = figure(x_axis_label='Store Number', y_axis_label='Average sales amount', width=600)
p3.circle(y='Avg_Sale_Amount', x=jitter('Store_Number', width=0.6), source=source, alpha=0.3)
p3.xaxis.major_tick_line_color = None  
p3.xaxis.minor_tick_line_color = None 
p3.xaxis.major_label_text_color = None   
p3.add_tools(hover)

In [11]:
df.Customer_Segment = pd.Categorical(df.Customer_Segment)
df['code'] = df.Customer_Segment.cat.codes

p4 = figure(x_axis_label='Customer Segment', y_axis_label='Average sales amount', width=600)
p4.circle(y='Avg_Sale_Amount', x=jitter('code', width=0.6), source=df, alpha=0.3)
p4.xaxis.major_tick_line_color = None  
p4.xaxis.minor_tick_line_color = None 
p4.xaxis.major_label_text_color = None  
p4.add_tools(hover)

In [12]:
row1 = column([p1, p3])
row2 = column([p2, p4])
layout = row([row1, row2])
show(layout)

By the look of the plots, customer segment and average number of product purchased are the only variables that impact average sales amount. I will go ahead and build a model with these two variables. I will be happy with an adjusted R^2 above 0.7

## Building the model

Here below I do a bit of data preparation, creating a dummies variables for the customer segment

In [13]:
df_with_dummies = pd.get_dummies(df, columns=['Customer_Segment'])
X = pd.DataFrame(data=df_with_dummies, columns=('Avg_Num_Products_Purchased', 'Customer_Segment_Credit Card Only',
 'Customer_Segment_Loyalty Club Only',
 'Customer_Segment_Loyalty Club and Credit Card',
 'Customer_Segment_Store Mailing List'))

X = X.dropna()

In [14]:
y = df[['Avg_Sale_Amount']]
y = y.dropna()

In [15]:
LR = sm.OLS(y, X).fit()

In [16]:
LR.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        Avg_Sale_Amount   R-squared:                       0.837
Model:                            OLS   Adj. R-squared:                  0.837
Method:                 Least Squares   F-statistic:                     3040.
Date:                Fri, 24 Aug 2018   Prob (F-statistic):               0.00
Time:                        11:03:34   Log-Likelihood:                -15061.
No. Observations:                2375   AIC:                         3.013e+04
Df Residuals:                    2370   BIC:                         3.016e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=================================================================================================================
                                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
Avg_Num_Products_Purchased                       66.9762      1.515     44.208      0.000      64.005      69.947
Customer_Segment_Credit Card Only               303.4635     10.576     28.694      0.000     282.725     324.202
Customer_Segment_Loyalty Club Only              154.1077      7.916     19.467      0.000     138.584     169.631
Customer_Segment_Loyalty Club and Credit Card   585.3022     14.823     39.487      0.000     556.235     614.369
Customer_Segment_Store Mailing List              58.0457      4.702     12.346      0.000      48.826      67.266
==============================================================================
Omnibus:                      359.638   Durbin-Watson:                   2.045
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4770.580
Skew:                           0.232   Prob(JB):                         0.00
Kurtosis:                       9.928   Cond. No.                         26.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The adjusted R^2 is abundantly above 0.7 which was my benchmark for ideal. The p-value of the coefficients are extremely small. All in all, I'm happy with the model

In [17]:
score_yes = mailing['Score_Yes'] # this will come in handy later

In [18]:
# target dataframe
mailing = pd.get_dummies(mailing, columns=['Customer_Segment'])
mailing = pd.DataFrame(data=mailing, columns=('Avg_Num_Products_Purchased', 'Customer_Segment_Credit Card Only',
 'Customer_Segment_Loyalty Club Only',
 'Customer_Segment_Loyalty Club and Credit Card',
 'Customer_Segment_Store Mailing List'))
mailing = mailing.dropna()

mailing.head()

,Avg_Num_Products_Purchased,Customer_Segment_Credit Card Only,Customer_Segment_Loyalty Club Only,Customer_Segment_Loyalty Club and Credit Card,Customer_Segment_Store Mailing List
0,3,0,1,0,0
1,6,0,0,1,0
2,7,0,1,0,0
3,2,0,1,0,0
4,4,0,1,0,0


In [19]:
forecast = LR.predict(mailing)
forecast.head()

0    355.036364
1    987.159466
2    622.941184
3    288.060159
4    422.012569
dtype: float64

Now I have the expected sales amount for each customer

## Calculation of the profit

Here I'm going to account for the probability of the customer to actually make a purchase (score_yes), the cost of a single catalog ($6.50) and the gross margin percentage of 50%

In [20]:
expected_revenue = (forecast*score_yes).sum()
print(expected_revenue)

47224.871373090646


In [21]:
profit = (expected_revenue * 0.5) - (250*6.5)
print(profit)

21987.435686545323


The expected profit from sending out the catalogs to the 250 new customers is $21,987.45